In [1]:
from importlib.util import find_spec
%load_ext autoreload
%autoreload 2

%matplotlib inline

if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('../..')

In [2]:
from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.models import QCNNSandwich
from tensorflow.keras.callbacks import ReduceLROnPlateau
import argparse
import numpy as np

import matplotlib.pyplot as plt
import time

2022-09-25 19:50:22.587054: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-25 19:50:22.587163: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-25 19:51:22.252800: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-25 19:51:22.252872: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bhagvada): /proc/driver/nvidia/version does not exist
2022-09-25 19:51:22.278151: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
T

In [15]:
args = argparse.Namespace()

# Data
args.center_crop = 0.7
args.resize = [8,8]
args.standardize = 1
args.binary_data = [0,1]
args.batch_size = 256
args.validation_split = 0.05
args.labels_to_categorical = 1

# Base Model
args.learning_rate = 1e-3
args.epochs = 10

# Quantum CNN Parameters
args.n_layers = 1
args.ansatz = 'NQubit'
args.sparse = 1
args.num_conv_layers = 1
args.conv_dims = [1]
args.num_qconv_layers = 1
args.qconv_dims = [1]

In [16]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Center cropping...
Center cropping...
Resizing data...
Resizing data...
Standardizing data...
Converting labels to categorical...
Converting labels to categorical...

Dataset :MNIST
╒════════╤══════════════════╤════════════════╤═════════════════╤═══════════╕
│ Data   │ Train size       │ Val size       │ Test size       │ Dims      │
╞════════╪══════════════════╪════════════════╪═════════════════╪═══════════╡
│ X      │ (12031, 8, 8, 1) │ (634, 8, 8, 1) │ (2115, 8, 8, 1) │ (8, 8, 1) │
├────────┼──────────────────┼────────────────┼─────────────────┼───────────┤
│ y      │ (12031, 2)       │ (634, 2)       │ (2115, 2)       │ (2,)      │
╘════════╧══════════════════╧════════════════╧═════════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪═══════╪════════╪═══════╪══════════════════════════╡
│ Train I

In [17]:
data.config()

{'input_dims': (8, 8, 1), 'output_dims': (2,), 'mapping': [0, 1]}

In [18]:
model = QCNNSandwich(data.config(), args)

../../qml_hep_lhc/layers/qconv2d.py:66: UserWarning: fm_class is only used for TwoLayerPQC.
  warnings.warn('fm_class is only used for TwoLayerPQC.')
../../qml_hep_lhc/layers/qconv2d.py:70: UserWarning: ansatz_class is only used for TwoLayerPQC.
  warnings.warn('ansatz_class is only used for TwoLayerPQC.')


In [19]:
model.compile()
model.build_graph().summary()

Model: "QCNNSandwich-AngleMap-NQubit"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 8, 8, 1)]         0         
                                                                 
 conv2d_4 (Conv2D)           (None, 8, 8, 1)           10        
                                                                 
 batch_normalization_2 (Batc  (None, 8, 8, 1)          4         
 hNormalization)                                                 
                                                                 
 qconv2d_0 (QConv2D)         (None, 8, 8, 1, 1)        18        
                                                                 
 flatten_36 (Flatten)        (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                      

In [20]:
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                              factor=np.sqrt(0.1),
                                              patience=5,
                                              min_delta=0.0001,
                                              min_lr=1e-8)
callbacks = [lr_scheduler_callback]
model.fit(data, callbacks)

Epoch 1/10
47/47 [==============================] - 177s 4s/step - loss: 0.2710 - auc_2: 0.9727 - accuracy: 0.9012 - val_loss: 0.1889 - val_auc_2: 0.9868 - val_accuracy: 0.9416 - lr: 0.0010
Epoch 2/10
47/47 [==============================] - 175s 4s/step - loss: 0.0741 - auc_2: 0.9972 - accuracy: 0.9798 - val_loss: 0.1797 - val_auc_2: 0.9824 - val_accuracy: 0.9353 - lr: 0.0010
Epoch 3/10
47/47 [==============================] - 175s 4s/step - loss: 0.0512 - auc_2: 0.9983 - accuracy: 0.9846 - val_loss: 0.0999 - val_auc_2: 0.9942 - val_accuracy: 0.9685 - lr: 0.0010
Epoch 4/10
47/47 [==============================] - 175s 4s/step - loss: 0.0431 - auc_2: 0.9985 - accuracy: 0.9865 - val_loss: 0.0496 - val_auc_2: 0.9975 - val_accuracy: 0.9890 - lr: 0.0010
Epoch 5/10
47/47 [==============================] - 175s 4s/step - loss: 0.0358 - auc_2: 0.9989 - accuracy: 0.9883 - val_loss: 0.0299 - val_auc_2: 0.9992 - val_accuracy: 0.9937 - lr: 0.0010
Epoch 6/10
47/47 [==============================] 

In [21]:
model.test(data, callbacks=callbacks)

9/9 [==============================] - 19s 2s/step - loss: 0.0259 - auc_2: 0.9996 - accuracy: 0.9915


[0.025947345420718193,
 0.9995562434196472,
 0.9914893507957458,
 0.9995562434196472]